In [1]:
import plotly.graph_objects as go

from ta.trend import MACD
from ta.momentum import StochasticOscillator

import numpy as np
import pandas as pd
from pykrx import stock
from pykrx import bond
from time import sleep

from datetime import datetime
import os
import time
from plotly.subplots import make_subplots
import glob

In [165]:

ticker_nm = '005930'
ticker_nm = '247540'
start_date  = '20190101'
today_date1 = '20231006'

df_raw = stock.get_market_ohlcv(start_date, today_date1, ticker_nm)
df_raw = df_raw.reset_index()
df_raw['ticker'] = ticker_nm

df_raw.columns = ['date', 'open', 'high', 'low', 'close', 'volume','price_change_percentage', 'ticker']

kor_stock_ohlcv_095570_total = df_raw


In [166]:
kor_stock_ohlcv_095570_total['MA5'] = kor_stock_ohlcv_095570_total['close'].rolling(window=5).mean()
kor_stock_ohlcv_095570_total['MA20'] = kor_stock_ohlcv_095570_total['close'].rolling(window=20).mean()
kor_stock_ohlcv_095570_total['MA60'] = kor_stock_ohlcv_095570_total['close'].rolling(window=60).mean()
kor_stock_ohlcv_095570_total['MA120'] = kor_stock_ohlcv_095570_total['close'].rolling(window=120).mean()

In [167]:

kor_stock_ohlcv_095570_total = kor_stock_ohlcv_095570_total[kor_stock_ohlcv_095570_total['date'] > '2023-01-01']
kor_stock_ohlcv_095570_total = kor_stock_ohlcv_095570_total.reset_index(drop = True)

In [168]:
kor_stock_ohlcv_095570_total['MA5-20'] = kor_stock_ohlcv_095570_total['MA5'] - kor_stock_ohlcv_095570_total['MA20']
kor_stock_ohlcv_095570_total['MA20-60'] = kor_stock_ohlcv_095570_total['MA20'] - kor_stock_ohlcv_095570_total['MA60']
kor_stock_ohlcv_095570_total['MA60-120'] = kor_stock_ohlcv_095570_total['MA60'] - kor_stock_ohlcv_095570_total['MA120']

/tmp/ipykernel_581499/2016690850.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_581499/2016690850.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_581499/2016690850.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
ascending_sq  = (kor_stock_ohlcv_095570_total['MA5-20'] > 0) & \
(kor_stock_ohlcv_095570_total['MA20-60'] > 0) & \
(kor_stock_ohlcv_095570_total['MA60-120'] > 0) 

descending_sq  = (kor_stock_ohlcv_095570_total['MA5-20'] < 0) & \
(kor_stock_ohlcv_095570_total['MA20-60'] < 0) & \
(kor_stock_ohlcv_095570_total['MA60-120'] < 0) 

In [171]:
ascending_intervals = []
i = 0
while i < len(ascending_sq):
    if ascending_sq[i]:
        start_index = i
        while i < len(ascending_sq) and ascending_sq[i]:
            i += 1
        end_index = i - 1
        ascending_intervals.append((start_index, end_index))

    else:
        i += 1

In [172]:
descending_intervals = []
i = 0
while i < len(descending_sq):
    if descending_sq[i]:
        start_index = i
        while i < len(descending_sq) and descending_sq[i]:
            i += 1
        end_index = i - 1
        descending_intervals.append((start_index, end_index))

    else:
        i += 1

In [198]:
# 골든 크로스 5-20
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_5_20_gold_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA5-20"].iloc[idx] > 0 and kor_stock_ohlcv_095570_total["MA5-20"].iloc[idx - 1] <= 0]
idx_5_20_gold_cross_df = pd.DataFrame({
    'index':idx_5_20_gold_cross,
    'name':'5-20 -><br> 골든크로스',
    'color':'red',
    'position':'top left'})


# 데드 크로스 5-20
# 양수에서 음수로 바뀌는 모든 인덱스 찾기
idx_5_20_dead_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA5-20"].iloc[idx] < 0 and kor_stock_ohlcv_095570_total["MA5-20"].iloc[idx - 1] >= 0]
idx_5_20_dead_cross_df = pd.DataFrame({
    'index':idx_5_20_dead_cross,
    'name':'5-20 -> <br>  데드크로스',
    'color':'blue',
    'position':'bottom left'})


# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_gold_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA20-60"].iloc[idx] > 0 and kor_stock_ohlcv_095570_total["MA20-60"].iloc[idx - 1] <= 0]
idx_20_60_gold_cross_df = pd.DataFrame({
    'index':idx_20_60_gold_cross,
    'name':'20-60 -> <br> 골든크로스',
    'color':'orange',
    'position':'top left'})


# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_20_60_dead_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA20-60"].iloc[idx] < 0 and kor_stock_ohlcv_095570_total["MA20-60"].iloc[idx - 1] >= 0]
idx_20_60_dead_cross_df = pd.DataFrame({
    'index':idx_20_60_dead_cross,
    'name':'20-60 -> <br>  데드크로스',
    'color':'purple',
    'position':'bottom left'})




# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_60_120_gold_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA60-120"].iloc[idx] > 0 and kor_stock_ohlcv_095570_total["MA60-120"].iloc[idx - 1] <= 0]
idx_60_120_gold_cross_df = pd.DataFrame({
    'index':idx_60_120_gold_cross,
    'name':'60-120 골든크로스',
    'color':'orange',
    'position':'top'})


# 골든 크로스 20-60
# 음수에서 양수로 바뀌는 모든 인덱스 찾기
idx_60_120_dead_cross = [idx for idx in range(len(kor_stock_ohlcv_095570_total)) if kor_stock_ohlcv_095570_total["MA60-120"].iloc[idx] < 0 and kor_stock_ohlcv_095570_total["MA60-120"].iloc[idx - 1] >= 0]
idx_60_120_dead_cross_df = pd.DataFrame({
    'index':idx_60_120_dead_cross,
    'name':'60-120 데드크로스',
    'color':'purple',
    'position':'bottom left'})

In [199]:
cross_df = pd.concat([idx_5_20_gold_cross_df, idx_5_20_dead_cross_df,
                      idx_20_60_gold_cross_df, idx_20_60_dead_cross_df])
cross_df = cross_df.reset_index(drop = True)

In [204]:
import plotly.express as px

# fig = go.Figure()
fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[1])

# 캔들스틱차트
fig.add_trace(go.Candlestick(
    x=kor_stock_ohlcv_095570_total['date'],
    open=kor_stock_ohlcv_095570_total['open'],
    high=kor_stock_ohlcv_095570_total['high'],
    low=kor_stock_ohlcv_095570_total['low'],
    close=kor_stock_ohlcv_095570_total['close'],
    increasing_line_color= 'red', decreasing_line_color= 'blue', 
    name = ''),
              row = 1, col = 1)

# MA 5 
fig.add_trace(go.Scatter(
    x=kor_stock_ohlcv_095570_total['date'],
    y=kor_stock_ohlcv_095570_total['MA5'],
    opacity=0.7,
    line=dict(color='blue', width=2),
    name='MA 5'))

# MA 20
fig.add_trace(go.Scatter(
    x=kor_stock_ohlcv_095570_total['date'],
    y=kor_stock_ohlcv_095570_total['MA20'],
    opacity=0.7,
    line=dict(color='orange', width=2),
    name='MA 20'))

# MA 60
fig.add_trace(go.Scatter(
    x=kor_stock_ohlcv_095570_total['date'],
    y=kor_stock_ohlcv_095570_total['MA60'],
    opacity=0.7,
    line=dict(color='purple', width=2),
    name='MA 60'))

# MA 120
fig.add_trace(go.Scatter(
    x=kor_stock_ohlcv_095570_total['date'],
    y=kor_stock_ohlcv_095570_total['MA120'],
    opacity=0.7,
    line=dict(color='green', width=2),
    name='MA 120'))

# 골든, 데드 크로스
for i in range(len(cross_df)):
    cross_index = cross_df['index'][i]
    cross_color = cross_df['color'][i]
    cross_name = cross_df['name'][i]
    cross_position = cross_df['position'][i]

    cross_date = kor_stock_ohlcv_095570_total['date'][cross_index]

    fig.add_vline(
        x=pd.to_datetime(cross_date).timestamp() * 1000,
        line_dash="dot",
        line_color = cross_color,
            annotation_text = cross_name,
            annotation_position = cross_position
    )
    
# 정배열, 역배열
# 정배열
for i in range(len(ascending_intervals)):
    start_date = ascending_intervals[i][0]
    end_date = ascending_intervals[i][1]

    fig.add_vrect(
        x0=kor_stock_ohlcv_095570_total['date'][start_date], 
        x1=kor_stock_ohlcv_095570_total['date'][end_date], 
        annotation_text=" ", annotation_position="top left",
                  fillcolor="red", opacity=0.1, line_width=0)
    
# 역배열
for i in range(len(descending_intervals)):
    start_date = descending_intervals[i][0]
    end_date = descending_intervals[i][1]

    fig.add_vrect(
        x0=kor_stock_ohlcv_095570_total['date'][start_date], 
        x1=kor_stock_ohlcv_095570_total['date'][end_date], 
        annotation_text=" ", annotation_position="top left",
                  fillcolor="blue", opacity=0.1, line_width=0)    
    
# Rayout
fig.update_layout(
    title = '삼성전자 주가',
    title_font_family="맑은고딕",
    title_font_size = 18,
    hoverlabel=dict(
        bgcolor='black',
        font_size=15,
    ),
    hovermode="x unified",
    template='plotly_dark',
    xaxis_tickangle=90,
    yaxis_tickformat = ',',
    legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
    barmode='group',
    margin=go.layout.Margin(
        l=10, #left margin
        r=10, #right margin
        b=10, #bottom margin
        t=100  #top margin
    ),
    height=400, width=900, 
#         showlegend=False, 
    xaxis_rangeslider_visible=False
)


# update y-axis label
fig.update_yaxes(title_text="Price", row=1, col=1)

fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])], row = 1, col = 1)
fig.show()

In [205]:
def ma_vis(data):
    # fig = go.Figure()
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True, vertical_spacing=0.01, row_heights=[1])

    # 캔들스틱차트
    fig.add_trace(go.Candlestick(
        x=kor_stock_ohlcv_095570_total['date'],
        open=kor_stock_ohlcv_095570_total['open'],
        high=kor_stock_ohlcv_095570_total['high'],
        low=kor_stock_ohlcv_095570_total['low'],
        close=kor_stock_ohlcv_095570_total['close'],
        increasing_line_color= 'red', decreasing_line_color= 'blue', 
        name = ''),
                  row = 1, col = 1)

    # MA 5 
    fig.add_trace(go.Scatter(
        x=kor_stock_ohlcv_095570_total['date'],
        y=kor_stock_ohlcv_095570_total['MA5'],
        opacity=0.7,
        line=dict(color='blue', width=2),
        name='MA 5'))

    # MA 20
    fig.add_trace(go.Scatter(
        x=kor_stock_ohlcv_095570_total['date'],
        y=kor_stock_ohlcv_095570_total['MA20'],
        opacity=0.7,
        line=dict(color='orange', width=2),
        name='MA 20'))

    # MA 60
    fig.add_trace(go.Scatter(
        x=kor_stock_ohlcv_095570_total['date'],
        y=kor_stock_ohlcv_095570_total['MA60'],
        opacity=0.7,
        line=dict(color='purple', width=2),
        name='MA 60'))

    # MA 120
    fig.add_trace(go.Scatter(
        x=kor_stock_ohlcv_095570_total['date'],
        y=kor_stock_ohlcv_095570_total['MA120'],
        opacity=0.7,
        line=dict(color='green', width=2),
        name='MA 120'))

    # 골든, 데드 크로스
    for i in range(len(cross_df)):
        cross_index = cross_df['index'][i]
        cross_color = cross_df['color'][i]
        cross_name = cross_df['name'][i]
        cross_position = cross_df['position'][i]

        cross_date = kor_stock_ohlcv_095570_total['date'][cross_index]

        fig.add_vline(
            x=pd.to_datetime(cross_date).timestamp() * 1000,
            line_dash="dot",
            line_color = cross_color,
                annotation_text = cross_name,
                annotation_position = cross_position
        )

    # 정배열, 역배열
    # 정배열
    for i in range(len(ascending_intervals)):
        start_date = ascending_intervals[i][0]
        end_date = ascending_intervals[i][1]

        fig.add_vrect(
            x0=kor_stock_ohlcv_095570_total['date'][start_date], 
            x1=kor_stock_ohlcv_095570_total['date'][end_date], 
            annotation_text=" ", annotation_position="top left",
                      fillcolor="red", opacity=0.1, line_width=0)

    # 역배열
    for i in range(len(descending_intervals)):
        start_date = descending_intervals[i][0]
        end_date = descending_intervals[i][1]

        fig.add_vrect(
            x0=kor_stock_ohlcv_095570_total['date'][start_date], 
            x1=kor_stock_ohlcv_095570_total['date'][end_date], 
            annotation_text=" ", annotation_position="top left",
                      fillcolor="blue", opacity=0.1, line_width=0)    

    # Rayout
    fig.update_layout(
        title = '삼성전자 주가',
        title_font_family="맑은고딕",
        title_font_size = 18,
        hoverlabel=dict(
            bgcolor='black',
            font_size=15,
        ),
        hovermode="x unified",
        template='plotly_dark',
        xaxis_tickangle=90,
        yaxis_tickformat = ',',
        legend = dict(orientation = 'h', xanchor = "center", x = 0.5, y= 1.2),
        barmode='group',
        margin=go.layout.Margin(
            l=10, #left margin
            r=10, #right margin
            b=10, #bottom margin
            t=100  #top margin
        ),
        height=400, width=900, 
    #         showlegend=False, 
        xaxis_rangeslider_visible=False
    )


    # update y-axis label
    fig.update_yaxes(title_text="Price", row=1, col=1)

    fig.update_xaxes(rangebreaks=[dict(bounds=["sat", "mon"])], row = 1, col = 1)
    fig.show()
    return fig




In [ ]:
ma_vis(kor_stock_ohlcv_095570_total)